## Construindo o modelo Keras para Redes Neurais Convolutivas

In [1]:
# Importações
import pandas as pd
from collections import OrderedDict
from sklearn.datasets import load_sample_images
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.feature import hog
import skimage
from skimage.io import imread
from skimage.transform import rescale
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import data, color
from skimage.feature import hog
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## A Ferramenta Keras

A ferramenta Keras é um framework desenvolvido em linguagem de programação Python que
suporta o desenvolvimento de modelos de Deep Learning.

O framework Keras (CHOLLET et al., 2015) é uma ferramenta para o desenvolvimento
de redes neurais profundas, contendo bibliotecas e métodos para cada um dos tipos
de redes descritas na literatura.
Estas redes podem ser programadas tanto em linguagem Python como em Linguagem R, pois o framework dá suporte a estas diferentes linguagens. A biblioteca é
aplicável nas seguintes condições:
- a) Prototipagem rápida e fácil (total modularidade, minimalismo e extensibilidade),
- b) Suporte a redes convolucionais e recorrentes, incluindo combinação de ambas,
- c) Suporte a esquemas de conectividade arbitrária (incluindo treino de N para N).
- d) Execução em CPU ou GPU.

Os modelos em Keras são definidos como uma sequência de camadas distintas em
redes neurais. Isto facilita a criação do modelo, bastando inserir uma camada por
vez até que estejamos satisfeitos com a topologia da rede. Primeiro, define-se a
camada de entrada com as dimensões corretas, como matrizes das imagens e vetores
de caracteristicas. Existe uma questão muito difícil a considerar, que é o número de
camadas e seus tipos. O Keras premite o uso de um método heurístico para encontrar
a melhor estrutura de rede, mas este algoritmo é custoso computacionalmente.


## Acesso aos dados

Os dados obtidos nos produtos HURSAT e GOES são especificamente dados matriciais. As imagens de eventos extremos correspondem a matrizes de 300x300 pixeis (ajuste padrão). Estas matrizes foram inseridas em uma tabela (Dataframe) para facilitar o acesso aos dados. Neste trabalho os Dataframes dividem as imagens em Eventos e Não-Eventos.

In [2]:
caminhoDataframe = "C:\/Users\/Administrador\/Desktop\/App Python\/"

def abreDataFrame(caminho,nomeArq):
    dataFrame = pd.read_pickle(caminho + nomeArq)#READ PICKLE MUITO IMPORTANTE!!!!!
    return dataFrame

df1 = abreDataFrame(caminhoDataframe, 'Data_Frame_EVENTO.pkl')
df2 = abreDataFrame(caminhoDataframe, 'Data_Frame_N_EVENTO.pkl')


In [3]:
df1.head(5)

,Array_IMG,Label
0,"[[[0.30812746], [0.33844912], [0.34922633], [0...",Evento
1,"[[[0.30812746], [0.33844912], [0.34922633], [0...",Evento
2,"[[[0.6184512], [0.6798183], [0.66341066], [0.6...",Evento
3,"[[[0.37978172], [0.39215672], [0.39215672], [0...",Evento
4,"[[[0.30333334], [0.30333334], [0.30190587], [0...",Evento


In [4]:
df2.head(5)

,Array_IMG,Label
0,"[[[0.15353093], [0.2007128], [0.1698009], [0.1...",N_Evento
1,"[[[0.15353093], [0.2007128], [0.1698009], [0.1...",N_Evento
2,"[[[0.056682266], [0.059096612], [0.059714362],...",N_Evento
3,"[[[0.059946757], [0.06290987], [0.06290987], [...",N_Evento
4,"[[[0.06433763], [0.076096654], [0.08731399], [...",N_Evento


## Rotulando as matrizes para corresponder aos objetos



In [6]:
'''
DADOS DA REDE NEURAL
'''
#ROTULANDO
def machine_learning_tag(dataframe):
    
    nome_valor = []
    for i in range(len(dataframe.index)):
        if(dataframe.loc[i,'Label'] == 'Evento'):
            valor = 1
            nome_valor.append(valor)
        elif(dataframe.loc[i,'Label'] == 'N_Evento'):
            valor = 0
            nome_valor.append(valor)
            
    dataframe['Tag_valor'] = nome_valor     
    
    return nome_valor

#Concatenar os DataFrames
CME_frames = [df1, df2]
dataFrame_CME = pd.concat(CME_frames)
#RESET INDEX -- 0,1,2,3,4, ... , N
dataFrame_CME = dataFrame_CME.reset_index(drop=True)


#Rotulando dados
#labels
nome_tag = machine_learning_tag(dataFrame_CME)
vetor_nome_tag = np.array(nome_tag)

## Dividindo os conjuntos de dados

A forma comum desta divisão do conjuntos de dados
obedece classicamente às seguintes proporções:

- a) Conjuntos de Treinamento: Correspondem a 80%, 66% ou 60% da amostra total de dados.

- b) Conjuntos de Teste e Validação: Correspondem a 20%, 33% ou 40% da amostra de dados quando não é necessário um conjunto de validação. Quando um conjunto de validação está presente o conjunto de teste pode corresponder à 10%, 15% ou 20% da amostra original. A tanto a validação quanto o teste não são submetidos ao modelo.

Esta forma de divisão dos dados de entrada propicia um cenário em que mesmo que
a convergência (aproximação de uma solução ótima) da RNA esteja em um nível
próximo ao mínimo ou máximo global, a resposta, ou seja, os testes e validações da
rede são executados em conjuntos com 80% menos dados do que a amostra original. Esta característica remove a representatividade da análise, sendo necessária a aplicação direta da rede para dados recém-coletados ou de outras bases.

In [7]:
#Dividindo Conjunto de treinamento e de Teste
X_train, X_test, y_train, y_test = train_test_split(
    dataFrame_CME.loc[:,'Array_IMG'],
    vetor_nome_tag,
    test_size=0.3,
    shuffle=True,
    random_state=42,
)
print(X_train.shape)


(1272,)


# Construindo modelo de Deep Learning

## Salvando os resultados e o modelo de treinamento da rede

In [8]:
'''
SALVANDO DICIONÁRIOS DE VALORES DOS TREINOS
'''
def salvaDicionarios(dicionarios,caminhoSaida,nomeArquivo):
    np.save(caminhoSaida+'\/'+nomeArquivo,dicionarios)

def carregaDicionarios(caminhoSaida,nomeArquivo):
    dicionarios = np.load(caminhoSaida+'\/'+nomeArquivo).item()
    return dicionarios

In [9]:
## Reshape das matrizes para se adequar às entradas da redes neurais

In [ ]:
'''
Ajusta vetores
'''
from keras.utils import to_categorical
from keras.models import Sequential

X_train = X_train.tolist()
X_test = X_test.tolist()

X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
'''
REDE CONVOLUTIVA - ARQUITETURA 1
'''
from sklearn.metrics import precision_score, recall_score

model2 = models.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 1)))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))

model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dense(2, activation='softmax'))

model2.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

history = model2.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

test_loss, test_acc = model2.evaluate(X_test, y_test)
print('test_acc:', test_acc)


#cria dicionario
history_dict = history.history
history_dict.keys()


import numpy
loss_history = history.history["loss"]
numpy_loss_history = numpy.array(loss_history)
numpy.savetxt("loss_history_Rede_Convolutiva_CME.txt", numpy_loss_history, delimiter=",")

#salva dicionário
salvaDicionarios(history_dict,caminhoDataframe,'Rede_Neural_Convolutiva_Extremos'+'.npy')

## Salvando o modelo treinado

In [ ]:
import pickle
# SALVANDO O MODELO
filename = "ARQ1_CONVNET_Extremos"+'.sav'
pickle.dump(model, open(filename, 'wb'))

## Carregando o Modelo

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

import pickle
def carregaModelo(nomeModelo):
    carrega_modelo = pickle.load(open(nomeModelo+'.sav', 'rb'))
    return carrega_modelo


model = carregaModelo("ARQ1_CONVNET_Extremos")

In [ ]:
history = carregaDicionarios(caminhoDataframe, 'Rede_Neural_Convolutiva_Extremos.npy')

acc = history['acc']
val_acc = history['val_acc']
loss = history['loss']
val_loss = history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc,'k', label='Acuracia no Treinamento')
plt.plot(epochs, val_acc, 'b', label='Acuracia no Teste')
plt.title('Acurácia no Treinamento e no Teste')
plt.legend()
plt.figure()
plt.plot(epochs, loss,'k', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Test loss')
plt.title('Erro no Treinamento e no Teste')
plt.legend()
plt.show()

plt.figure()
acc_values = history['acc']
val_acc_values = history['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


## Gerando Matriz de Confusão


In [ ]:
Y_prediction = model.predict(X_test)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_prediction,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 

plt.figure(figsize=(10,8))
sns.heatmap(confusion_mtx, annot=True, fmt="d");

## Gerando Ativação das Camadas

In [ ]:
'''
A ativação das Camadas e suas Imagens - Resposta do Que a Rede Abstrai no decorrer
dos Treinamentos.
'''
from keras.models import Model
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(X2[620].reshape(1,300, 300,1))
 
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index])
            activation_index += 1

plt.figure()
plt.imshow(X2[620][:,:,0]);

display_activation(activations, 5, 4, 1)

display_activation(activations, 5, 4, 2)

display_activation(activations, 5, 4, 3)

display_activation(activations, 5, 4, 4)